1 - Objetivo do Modelo

2 - Como obteve os Dados e quais informações temos sobre esse dataset?

3 - Quais as etapas você usou para modelagem (como escolheu o modelo, quais metricas usou, o que fez para processar os dados) ?

4 - Resultados experimentais da modelagem (quanto acerta? Falsos positivos? Verdadeiros Negativos?)

# Imports

In [ ]:
!pip3 install snscrape
!pip install spacy_cleaner
!pip install syllables
!pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
ERROR: pip's dependency resolver do

In [ ]:
import re
import spacy
import string
import spacy_cleaner
import demoji
import pandas as pd 
import numpy as np
import snscrape.modules.twitter as SnTwitter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

# palavrões

In [ ]:
palavrao = [
  'Arrombado', 'Babaca', 'Baitola', 'Bicha', 'Bosta', 'Buceta', 'Cacete', 'Caralho', 'Cu', 'DebilMental', 'Demente', 'Desgracado', 'Enfianocu', 'Escroto', 'FDP', 'Foda', 'Fuder', 'Fudido', 'fudeo', 'fodasse',
 'Gozado', 'Idiota', 'Imbecil', 'Imundo', 'Ku', 'Limpezaanal', 'lixo',
 
 'MiolodeCu', 
 'MontedeMerda',
 'Olhodocu',
 'Otario', 'Palhaco',




 
 'Piroca',
 'Porra',
 'Punheta', 'Puta', 'PutaQuePariu', 'pqp'
 
 'Rapariga',
 'Retardado',
 

 'Vadia', 'Vagabundo', 'vagabunda',
 
 'Viado', 
 'Xereca',
 
 
 'arrombado', 
 'baba-ovo', 'babaca', 'babaovo',
 'baitola', 
 'bicha', 'bixa',
 'boceta', 'buceta', 
 'boiola', 'bokete', 
 'boquete', 'bosseta', 'bosta', 
 'buceta', 
 
 'caceta', 'cacete', 
 'canalha',
 'caralho', 'casseta', 'cassete', 
 'checheca', 'chereca',
 'chifruda', 'chifrudo', 
 'corna', 'cornagem', 'cornisse', 'corno', 'cornuda', 'cornudo', 'cornão',
 'cú',
 'curalho', 'cuzao', 'cuzão', 'cuzuda', 'cuzudo', 
 'egua', 'égua', 
 'estupida', 'estúpida', 'estupidez', 'estupido', 'estúpido',
 'foda', 'fodao', 'fodão', 'fode', 'fodi', 'fodida', 'fodido',
 'fudendo', 'fudeção', 'fudida', 'fudido', 
 'iscrota', 'iscroto', 
 'masturba', 
 'merda',
 'mongoloide', 'mongolóide',


 'pica', 'picao', 'picão', 
 'pinto', 'pintudo', 'pintão', 
 'piroca',


 'porra',
 'punheta', 'punhetao', 'punhetão',
 'puta', 'puto',
 'retardada', 'retardado', 'siririca', 'vadia', 'vagabunda', 'vagabundo', 'vagina', 'veadao', 'viada', 'viadagem', 'viadao', 'viadão', 'viado', 'viadão', 'víado', 'xerereca', 'xexeca',
]
translator = str.maketrans('','',string.punctuation)

for i,c in enumerate(palavrao):
    palavrao[i] =c.translate(translator)
    palavrao[i] = c.lower()


a = 'Animal de Teta'
print(a.lower())


print(palavrao)

animal de teta
['arrombado', 'babaca', 'baitola', 'bicha', 'bosta', 'buceta', 'cacete', 'caralho', 'cu', 'debilmental', 'demente', 'desgracado', 'enfianocu', 'escroto', 'fdp', 'foda', 'fuder', 'fudido', 'fudeo', 'fodasse', 'gozado', 'idiota', 'imbecil', 'imundo', 'ku', 'limpezaanal', 'lixo', 'miolodecu', 'montedemerda', 'olhodocu', 'otario', 'palhaco', 'piroca', 'porra', 'punheta', 'puta', 'putaquepariu', 'pqprapariga', 'retardado', 'vadia', 'vagabundo', 'vagabunda', 'viado', 'xereca', 'arrombado', 'baba-ovo', 'babaca', 'babaovo', 'baitola', 'bicha', 'bixa', 'boceta', 'buceta', 'boiola', 'bokete', 'boquete', 'bosseta', 'bosta', 'buceta', 'caceta', 'cacete', 'canalha', 'caralho', 'casseta', 'cassete', 'checheca', 'chereca', 'chifruda', 'chifrudo', 'corna', 'cornagem', 'cornisse', 'corno', 'cornuda', 'cornudo', 'cornão', 'cú', 'curalho', 'cuzao', 'cuzão', 'cuzuda', 'cuzudo', 'egua', 'égua', 'estupida', 'estúpida', 'estupidez', 'estupido', 'estúpido', 'foda', 'fodao', 'fodão', 'fode', 'fo

#scrapping dos dados (não a necessidade de rodar, apenas fassa o upload dos dado)

In [ ]:
# palavrao = re.compile(r'((?i)\bm[e3]rd\S+\b)|(?i)\bb[o0]st\S+\b|(?i)\bput[^r]\S*\b|(?i)\bp[o0]rr[a@4]\S*\b|(?i)\bc[uú](\b|z\w+)|(?i)\bv[i1](nh)?[a@4]d[^u]\S*\b|(?i)\b[kc][o0a@4]?r[o0a@4]?l[hi]?[o0a@4]?\S*\b|(?i)\bf[o0u]d\S+\b|\b[fF](\.)?[dD](\.)?[pP]\b|\b[Pp](\.)?[Qq](\.)?[pP]\b|\b[Vv][Ss][Ff]\b|\b[Vv][Tt][Nn][Cc]\b|(?i)b[o0u]c[e3]t\S+\b|(?i)\bpunh[e3]t\S+\b|(?i)\bb[i1](ch|x)[a@4]s?\b|\b(?i)c[o0]c[0ô]s?\b|\b(?i)[e3]scr[o0]t\S+\b|(?i)\bb[a@4]b[a@4][qc]\S+\b|(?i)\bc[a@4]g\S+\b|(?i)\bs[a@4]c[a@4]n[a@4e31i]\S*\b|(?i)\bk[a@4]c[e3]?t[e3]?\S*\b')



scrapper = SnTwitter.TwitterSearchScraper('lang:pt')



x = []
y = [] 

c_palavrão = 0 
c_sem_palavrão = 0
for i, tweet in enumerate(scrapper.get_items()):
  print(c_palavrão, c_sem_palavrão)
  if c_palavrão > 9000 and c_sem_palavrão > 10000:
    break

  new_tweet = tweet.rawContent.split(' ')


      
  new_tweet = new_tweet.replace('\n', '')
  new_tweet = re.sub(r'\B#\w+', '', new_tweet)#hastag
  new_tweet = demoji.replace(new_tweet, '') #emojis
  new_tweet = re.sub(r'http\S+', '', new_tweet)#links

  # print(new_tweet)
  zx = False
  for c in new_tweet.split():
    # print(new_tweet.split())
    if c in palavrao:

       zx = True
  if zx: #o regex não inclue essa palavra
    print('oi')
    if c_palavrão > 9000:
      continue


      
    x.append(new_tweet.lower())
    y.append(1)
    c_palavrão +=1
  
  else:
    if c_sem_palavrão > 10000:
      continue
    if tweet.lang != "pt" and tweet.lang != 'pt-br':
      continue

    x.append(new_tweet.lower())
    y.append(0)
    c_sem_palavrão +=1
print(x)





ERROR:snscrape.base:Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=lang%3Apt&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&include_ext_edit_control=true&ext=mediaStat

ScraperException: ignored

In [ ]:
# d = {'frases': x, 'palavrão': y}
# df = pd.DataFrame(data=d)
# df.to_csv('output.csv')


df = pd.read_csv('/content/df_palavroesv2.csv')

# df = pd.concat([df1, df2], ignore_index=True)


#  preprocessamento

In [ ]:
#roda se o erro 
#"NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968" 
#aparecer durante a instalação do pt_core_news_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy download pt_core_news_sm

2023-05-29 13:17:23.619802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


Remove o vies dos dados:

In [ ]:
df['frases'].isnull().sum()
df =df.dropna()

In [ ]:
import pt_core_news_sm
#substitue o nome lulaoficial por nomes aleteatorios e cria uma nova coluna com a pessoa da frase
nlp = pt_core_news_sm.load()
for i,c in enumerate(df['frases']):
  # print(i)
  if 'lulaoficial' in c or 'lula' in c or 'brasil' in c:
    df.frases[i] = df.frases[i].replace('lulaoficial', np.random.choice(["Alice", "Breno", "Carmem", "Dalva", "Edson", "Felipe", "Gabriela", "Henrique", "Igor", "Júlia", "Karla", "Lara", "Márcio", "Nina", "Olívia", "Pablo", "Ricardo", "Sara", "Tânia", "Ubirajara", "Valentina", "Wagner", "Xavier", "Yago", "Zélia"]))
    df.frases[i] = df.frases[i].replace('lula', np.random.choice(["Alice", "Breno", "Carmem", "Dalva", "Edson", "Felipe", "Gabriela", "Henrique", "Igor", "Júlia", "Karla", "Lara", "Márcio", "Nina", "Olívia", "Pablo", "Ricardo", "Sara", "Tânia", "Ubirajara", "Valentina", "Wagner", "Xavier", "Yago", "Zélia"]))
    df.frases[i] = df.frases[i].replace('Brasil', np.random.choice(["Angola", "Benin", "Brasil", "Chade", "Cuba", "Dinamarca", "Fiji", "Gana", "Haiti", "Índia", "Japão", "Jordânia", "Kuwait", "Lesoto", "Malta", "México", "Moçambique", "Níger", "Panamá", "Paraguai", "Peru", "Polônia", "Qatar", "Ruanda", "Síria", "Sudão", "Suriname", "Tunísia", "Turquia", "Uganda", "Uruguai", "Vanuatu"]
))
    


<ipython-input-9-b99c229d918d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.frases[i] = df.frases[i].replace('lula', np.random.choice(["Alice", "Breno", "Carmem", "Dalva", "Edson", "Felipe", "Gabriela", "Henrique", "Igor", "Júlia", "Karla", "Lara", "Márcio", "Nina", "Olívia", "Pablo", "Ricardo", "Sara", "Tânia", "Ubirajara", "Valentina", "Wagner", "Xavier", "Yago", "Zélia"]))
<ipython-input-9-b99c229d918d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.frases[i] = df.frases[i].replace('Brasil', np.random.choice(["Angola", "Benin", "Brasil", "Chade", "Cuba", "Dinamarca", "Fiji", "Gana", "Haiti", "Índia", "Ja

In [ ]:
import string
class PLNprocess():
  def __init__(self):
    self.pln = pt_core_news_sm.load()
    self.stop_words = self.pln.Defaults.stop_words
    self.pontuacoes = string.punctuation
    return

  def preprocessamento(self,text):
    result = []
    
    texto = re.sub(r'[^a-zA-z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙÃãõÕçÇ: ]','',text)
    doc = self.pln(texto.lower())
    
    for token in doc:
      
      if(token.text in self.stop_words or token.text in self.pontuacoes ):
        continue
      
      
      result.append(token.lemma_)
      

    
    texto = ' '.join([str(elemento) for elemento in result if not elemento.isdigit()])
    return texto



In [ ]:
process = PLNprocess() 
for i, c in enumerate(df['frases']): 
  textoProcessado = process.preprocessamento(c) 
  df['frases'][i] = textoProcessado 

df['frases'] 


<ipython-input-11-d6e9ee60877f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frases'][i] = textoProcessado
<ipython-input-11-d6e9ee60877f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frases'][i] = textoProcessado


0                          cruzeiro ggaleixo zagueiro em é
1                          adotar apartir agr agenda lotar
2                                          amo fzr redação
3        vontade sair existemal ficar casa ver série se...
4                           dudacastrx mandar amg esquecer
                               ...                        
14002                                         foder porrar
14003            vtxzn porra tá malucooo larica pprt kkkkk
14004    thaidoback não sinto minhas pernas thaimeu joe...
14005                                       se foder porra
14006     vtxzn porra tá malucooo nem na larica pprt kkkkk
Name: frases, Length: 14004, dtype: object

In [ ]:
import pickle
vectorizer = TfidfVectorizer()
vectorizer.fit(df['frases'])
with open('vectorinizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# teste de algoritimos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x = df['frases']
y = df['palavrão']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state = 0)

x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)



x_train = x_train.astype(np.float32)
x_test =x_test.astype(np.float32)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

LGBM = LGBMClassifier(learning_rate =0.1, num_leaves = 100)
svm = SVC(C=20, random_state=0)
florest = RandomForestClassifier(200, random_state=0, criterion = 'entropy', max_features='log2')
regression = LogisticRegression(penalty = 'l1',C=7.5,solver = 'liblinear',random_state=0) 

LGBM.fit(x_train, y_train)
florest.fit(x_train, y_train)
regression.fit(x_train, y_train)
svm.fit(x_train, y_train)

previsoes_f = florest.predict(x_test)
previsoes_r = regression.predict(x_test)
previsoes_s = svm.predict(x_test)
previsoes_l = LGBM.predict(x_test)

print(f"floresta  {accuracy_score(y_test, previsoes_f)}, regressão logistica: {accuracy_score(y_test, previsoes_r)} svm: {accuracy_score(y_test, previsoes_s)} LGBM: {accuracy_score(y_test, previsoes_l)} ")

print(classification_report(y_test, previsoes_r))



# Interferencia

In [ ]:
frasesInferencia = {

    'Eu vou a escola':0,
    'De acordo com a constituição de 1988':0,
    'eu te amo':0 ,
    'hoje eu tava andando e caiu um pedasso de pau na minha cabeça':0,
    'qual teclado compensa mais um redragon ou um razer':0,
    'oi eai como você está':0,
    'oi, você que ir no açai hoje':0,
    'dessa forma podemos concluir que cada vez mais termos mudanças na educação':0,
    'apesar de aparentem extremamente potentes, computadores quanticos não resolveram todos os nossos problemas':0,
    'por que que para eles funcionaram precisam estar perto do 0 absoluto':0,
    'para que não haja interferencia nas particulas':0,
    'mas ainda sim não ta errado isso':0,
    'eu acho que podemos fazer melhor':0, 
    'ele fazia bulling comigo':0,
    'o manual desse rologio é muito bem feito':0,
    'tava pensando em virar lider de turma so pelo presente':0,
    'meu animal favorito é lagosta, e o seu':0,
    'meu lapiz novo é muito bom':0,
    'essa prova tava muito chata':0,
    'meu computador começou a travar do nada':0,
    'eu amo luz rgb':0,
    'namora comigo':0,
    'aque cara é muito chato':0,
    'se eu fosse você não dava moral pra isso':0,
    'não fica assim, vai dar certo':0,
   
    'Vai se foder':1,
    'podemos concluir que os games estarão cada vez mais presentes na educação':0,
    'eu adoro sorvete. ai hoje quando eu tava na frente da padaria eu fiquei puto com preço':1,
    'Esse politicos vagabundo tem de serem presos':1,
    'A puta que pariu que essa porra é assim':1,
    'namoral vai tomar no cu':1,
    'adiciona ele não, esse cara é muito fudido': 1,
    'se fosse eu mandava metia o fodasse e ia pra proxima':1,
    'se aquele viado não calar a boca eu vo ate ali dar um murro nele':1,
    'era pra isso tar pronto hoje seu vagabundo': 1,
    'para de foder com o trabalho':1,
    'que merda, bati o dedinho na porta':1,
    'caralho ta bem mais caro':1,
    'ele disse: toma no cu':1,
    'você não faz ideia de como eu odeio aquele viado':1,
    'Esse cara e chato pra caralho':1,
    'essa puta fica andando por ai com a calsa no cu':1,
    'pora você fudeo com tudo':1,
    'você ta fudido':1,
    'compra logo essa porra':1,
    'que buceta':1,
    'para de andar com esses vagabundos, vai acabar com a sua reputação':1,
    'se cabelo fosse importante não nascia no cu':1,
    'seriao depois dessa você ta bem fudido':1,
    'merda, esqueci o dever':1
    }


l = []
pre = []
for frase,label in frasesInferencia.items():
    frase = process.preprocessamento(frase)
    vector = vectorizer.transform([process.preprocessamento(frase)])
    p = regression.predict(vector)
    l.append([label])
    pre.append(p)




print(classification_report(l, pre))
print(accuracy_score(l, pre))



In [ ]:
#Frases que ele errou:
for frase,label in frasesInferencia.items():
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  if label != predicao:
    print(f'Na frase: \n {frase} \nVALOR REAL: {label}\nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')




In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(l, pre, labels=regression.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                        display_labels=regression.classes_)
disp.plot()

Matrix de confusão do df de teste

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_test, previsoes_r, labels=regression.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                        display_labels=regression.classes_)
disp.plot()

In [ ]:

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def plot_roc_curve(modelo, X_teste, y_teste):
    # Obtém as probabilidades previstas e os valores verdadeiros
    y_pred_proba = modelo.predict_proba(X_teste)[:, 1]
    y_true = y_teste

    # Calcula a curva ROC
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # Plota a curva ROC
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.title('Curva ROC')
    plt.legend(loc="lower right")
    plt.show()
plot_roc_curve(regression, x_test, y_test)



# Análise exploratória do dataframe



In [ ]:
import seaborn as sns
sns.countplot(x='palavrão',data=df)

Quantidade ralativamente igual de palavras com e sem palavrões

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator, STOPWORDS
import matplotlib.pyplot as plt
stop = pt_core_news_sm.load().Defaults.stop_words
stop.add('pra')
stop.add('ão')
wordcloud = WordCloud(stopwords=stop).generate(' '.join(df.frases.values.tolist()))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


# Teste em redações

In [ ]:
a = 'Querido filho,\n Fiquei sabendo que você terá seu primeiro debate fudido na próxima semana. Eu já conheço bem suas ideias antifeministas e sei que a feminista que debaterá com você é extremista. Sendo seu pai, sinto que é meu dever te lhe dar alguns conselhos.\n Quando for debater, mantenha a calma e não ataque a argumentadora oponente, mas sim seus argumentos. Esse é o primeiro passo para que o debate seja civilizado e produtivo. Use a razão e a lógica. Caso a feminista faça o oposto disso, e, por exemplo, descarte os seus argumentos meramente por você ser homem, branco, heterossexual e cristão, explique ao público que ela está usando a famosa falácia ad hominem.\n Evite e aponte, meu querido filho, todas as falácias lógicas. Falácias destroem o bom debate. Caso a feminista se esquive das críticas contra o feminismo, afirmando que aquilo que está sendo criticado por você chama-se "femismo" e não "feminismo", avise-a que ela está utilizando a falácia do escocês de verdade.\n  \n Mantenha-se firme também na defesa pela liberdade de expressão. Essa é outra regra de ouro para o bom debate. Não importa o quão verbalmente agressiva seja a feminista ou quão repulsivas pareçam ser suas ideias, lembre-se que ela possui o direito de falar tudo aquilo que realmente pensa. Se tudo o que ela fala for de fato tão horrível e irracional assim, então será fácil para você desconstruir publicamente os argumentos dela. Lembre-se daquilo que sempre te ensinei: tentar calar o debatedor adversário à força, como se estivesse cometendo um crime por expor o que pensa, é para aqueles que não conseguem convencer os demais através do uso da lógica e da razão, então precisam fazer isso através de meios coercitivos.\n Portanto, filho, se alguém nesse debate utilizar constantemente argumentos falaciosos ou tentar calar à força o adversário, alegando estar combatendo um "discurso de ódio", então que esse alguém não seja você. Não podemos controlar a feminista com quem você debaterá, mas você tem controle sobre si próprio. Faça a sua parte e tenha um bom debate!\n Com carinho,\n Seu amado Pai.'
a = a.replace('\n', '')
a = a.split('.')
a = [i.strip() for s in a for i in s.split(',')]

for frase in a:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')







In [ ]:
b = 'O conhecimento é fundamental para que haja uma opinião crítica sobre um determinado assunto, esse é o ponto de partida de um argumento num debate. Sobretudo há um grande problema com grande parte da população brasileira, que é a falta de leitura.\n O Brasil é um país com uma baixa taxa de leitores, de acordo com pesquisas, muito dos brasileiros tem deixado os livros em segundo plano, isso porque o uso de celulares, tabletes e notebooks são os maiores concorrentes com o popular e conhecido livro.\n A leitura proporciona, além do conhecimento, novos horizontes para que as pessoas possam ter opiniões formadas e entender sobre diversos assuntos. Assim num debate, ou numa discussão, não haverá aquele velho discurso que todos usam e que ninguém nem sabe o que realmente significa, mas que compartilham em redes sociais como se fossem os próprios autores.\n Um debate que gera resultados satisfatórios é feito por pessoas com habilidades adquiridas através da leitura, da busca pelo conhecimento e pela criticidade, debater não significa brigar, é modo de cada um mostrar com argumentos válidos seu posicionamento, é uma conversa que pode gerar controvérsias, mas que é aceita e no final agrega sabedoria.'
b = b.replace('\n', '')
b = b.split('.')
b = [i.strip() for s in b for i in s.split(',')]

for frase in b:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')







In [ ]:
c = 'No âmbito escolar, a discriminação vem gerando cada vez mais atos de violências devido a diversidade de opções sexuais. A partir destes aspectos os preconceitos se originam, mas o respeito é direito e dever de toda a população.\n O preconceito dos alunos com os outros que optam por uma escolha sexual diferente é perceptível, como consequência seguinte a violência não sendo ela apenas física, também sentimental e etc. É frequente a decorrência de fatos de discriminação por esse motivo, como exemplo podemos citar inúmeros casos, que já foram retratados nas telenovelas da Rede Globo onde pessoas homossexuais foram vítimas de preconceito e discriminação.\n Segundo a Constituição Brasileira, todos são iguais perante a lei e todos os brasileiros e estrangeiros residentes tem direito à vida, à liberdade, à igualdade, à segurança e à propriedade sem separação de qualquer natureza.\n Portanto, sendo um problema á sociedade, as rejeições estão presentes no nosso cotidiano escolar. Necessitando assim, de uma criação de programas sobre o direito e dever dos alunos de respeitar as diferenças sexuais de seus colegas, contando com o apoio da secretaria municipal de educação para desenvolver palestras educativas, os pais pais também precisam estar colaborando para que os resultados sejam efetivos gerando assim uma convivência pacifica dentro e fora do ambiente escolar.'
c = c.replace('\n', '')
c = c.split('.') 
c= [i.strip() for s in c for i in s.split(',')]

for frase in c:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')







In [ ]:
d = """O aquecimento global vem afetando o mundo nos últimos anos, ess fator e algo natural porem está aumentando drasticamente pela quantidade de combustíveis quem são queimados diariamente, assim afetando o ambiente onde vivemos, ocasionando uma elevação na temperatura ambiente.\n A partir disso surgem vários problemas que afetam o habitat natural de vários animais, e um deles e o derretimento das geleiras que obrigam os animais como o urso polar a mudar seus hábitos de caça.\n Muitos ursos para se alimentarem começaram a invadir cidades, a relatos de que os ursos atacam pessoas para se alimentar. Muitos estão pensando na hipótese de caça aos ursos porem esse método e proibido em lei, além disso, a agência do Meio Ambiente russa negou a licença para abater essa espécie pois a mesma se encontra em extinção.\n Portanto, medidas são necessárias para resolver esse impasse podendo ter algum órgão governamental responsável pelo meio ambiente para se responsabilizar de resgatar e alimentar esses animais para evitar futuros ataques contra pessoas. A Rússia o quinto pais do ranking de emissores de queima de combustíveis para que ela possa reduzir a queima de combustíveis deve investir em um combustível mais "limpo" para o ambiente por exemplo o etanol além de estimular o uso do mesmo"""
d = d.replace('\n', '')
d = d.split('.')
d = [i.strip() for s in d for i in s.split(',')]

for frase in d:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')





In [ ]:
e = """'Em dezembro de 2019, o mundo foi acometido por um vírus de alarmante potencial pandêmico (oriundo de Wuhan, na China), que rapidamente espalhou-se por todo o globo terrestre. Gerando impactos sociais, como a restrição da circulação Humana, e a pausa dos trabalhos não essências para a sobrevivência. O que também levou á impactos ambientais, porém pouco duradouros.\n \n \n \n Devido a paralisação de atividades econômicas, a qualidade do ar teve uma melhora notória, o que acarreta diversos benefícios ao meio ambiente e consequentemente a nossa Saúde. Doenças respiratórias como: Renite, Asma, pneumonia, câncer de pulmão e etc. São em parte, consequências do estado deplorável do ar. No entanto, tais mudanças não são permanentes, tendo em consideração que as mudanças de hábitos se devem as restrições sociais impostas pela Organização Mundial de Saúde (OMS), e que os esforços para a recuperação do estado econômico do país pós pandemia, serão Extremamente prejudiciais, considerando que a produção de poluentes como o CO2 serão exacerbadas.\n \n \n \n No final do ano de 2019, a Organização das Nações Unidas (ONU) afirmou que para conter catástrofes naturais, seria necessário uma diminuição de 7,6% de emissão de carbono por 10 anos. O que de fato é impossível considerando-se que o Brasil, assim como tantos outros países, está em um processo de desenvolvimento que demanda uma alta atividade industrial. Mesmo com os esforços feitos em meio a pandemia, houve a diminuição de apenas 5% de carbono emitido no ar.\n \n Por essas considerações, é nítido que os impactos ambientais em meio a Pandemia, nos traz falsas esperanças de melhoras."""
e= e.replace('\n', '')
e= e.split('.')
e = [i.strip() for s in e for i in s.split(',')]

for frase in e:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')




In [ ]:
f  = """'Após as Revoluções Industriais o homem passou exigir mais do meio-ambiente utilizando seus recursos e poluindo-o em níveis alarmantes, tudo isso para manter a demanda de consumo que com o passar dos anos foi aumentando. Inesperadamente surge uma doença, a Covid-19, que posteriormente é classificada como pandemia. As indústrias do mundo todo tiveram que interromper suas atividades, o que acabou impactando positivamente o planeta na questão ambiental. \n \n As pandemias sempre foram presentes no decorrer da história, mas nenhuma delas até então havia ocorrido em um mundo tão globalizado como o que vivemos, com isso o poder de disseminação da doença é avassalador, o que consequentemente torna seus impactos maiores e mais preocupantes. Isso obrigou a população mundial a mudar os hábitos e trouxe mudanças inimagináveis. \n \n No Brasil, grandes cidades como São Paulo se viram forçadas a traçar planos a fim de conter o avanço do vírus, utilizando- se de medidas que antes jamais haviam sido cogitadas, mas que foram necessárias. Com isso o tempo que as pessoas passam em casa aumentou e o uso de automóveis diminuiu, fazendo com que a emissão de seus gases se reduzissem, dando um “alívio” à atmosfera. \n \n  Em virtude dos fatos apresentados, somos levados a acreditar que o padrão de vida atual é uma enorme ameaça à saúde do meio- ambiente. Sendo assim, é fundamental que órgão federais como o Ministério do Meio-Ambiente adotem posturas que incentivem a fiscalização dessas indústrias e puna quem desrespeitar as normas (respeitando os Diretos Humanos). Junto disso, se faz necessário que atitudes também sejam tomadas pelo Ministério da educação (MEC), para que as pessoas tenham o senso ecológico e a responsabilidade socioambiental aflorados desde a pré-escola. A esperança de um povo mais responsável e um planeta mais saudável, se torna mais real com cada uma fazendo sua parte.'"""

f= f.replace('\n', '')
f= f.split('.')
f = [i.strip() for s in f for i in s.split(',')]

for frase in f:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')


In [ ]:
g  = """'O conhecimento é fundamental para que haja uma opinião crítica sobre um determinado assunto, caralho, esse é o ponto de partida de um argumento num debate. Sobretudo há um grande problema com grande parte da população brasileira, que é a falta de leitura. \n O Brasil é um país com uma baixa taxa de leitores, de acordo com pesquisas, todos os brasileiros tem deixado os livros em segundo plano, isso porque o uso de celulares, tabletes e notebooks são os maiores concorrentes com o popular e conhecido livro. \n  A leitura proporciona, além do conhecimento, novos horizontes para que as pessoas possam ter opiniões formadas e entender sobre diversos assuntos. Assim num debate, ou numa discussão, não haverá aquele velho discurso que todos usam e que ninguém nem sabe o que realmente significa, mas que compartilham em redes sociais como se fossem os próprios autores. \n  Um debate que gera resultados satisfatórios é feito por pessoas com habilidades adquiridas através da leitura, da busca pelo conhecimento e pela criticidade, debater não significa brigar, é modo de cada um mostrar com argumentos válidos seu posicionamento, é uma conversa que pode gerar controvérsias, mas que é aceita e no final agrega sabedoria.'"""


g= g.replace('\n', '')
g= g.split('.')
g = [i.strip() for s in g for i in s.split(',')]

for frase in g:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')


In [ ]:
h  = """Umas das causas que mais têm se visto é machismo que leva à um morte de uma mulher , pode ser término de relacionamento ou não, ciúmes, possesividade, achar que dona dela que essa não tem vida so que eles querem um outro ponto a ser levado em consideração é quanto já foi sofrido antes do crime acontecer, será ja tentaram ajuda para se livrar ,leis que podiam ajudar mas nem sempre tem sua eficaz quando precisa-se como a Lei Maria da Penha, tem como ajudar para manter longe esse agressor nem sempre é suficiente é que vemos muito no jornal hoje mais caso de feminicidio,amanha e depois. \n Uma campanha so do governo também não é suficiente para alertar mulheres que vêm pasando por violencia pois para que essas não sejam mortas como outras tantas tem vir acompanhada de tratamento psicologicos acompanhado de assistência para saibam que não estão sozinha ée todas possam ser ajudadas antes quemais mortes aconteçam.'"""


h= h.replace('\n', '')
h= h.split('.')


for frase in h:
  vector = vectorizer.transform([process.preprocessamento(frase)])
  predicao = regression.predict(vector)
  proba = regression.predict_proba(vector)
  
  print(f'Na frase: \n {frase} \nVALOR PREDITO: {predicao} com {round(max(proba[0])*100,2)}% de probabilidade\n\n')


# Explicação tecnica

**Quantidade de dados** foi de 14 mil 10 sem palavrão e 1 sem palavrão pois dessa forma criamos uma tendencia no modelo a prever 0 o que evita falsos positivos além de que no mundo real a maior parte das redações não contem palavrões.
Os valores 10k e 4k foram escolidos com base em testes. Mais que isso faz uma diferença muito pequena o que não justifica o esforço computacional e uma quantidade menor do que 4k para 1 não é suficiente para o modelo aprender todos os palavrões.



**Preprocessamento:** Fiz a remoção de puntuação e stop words por não serem relevantes a classe ( cheguei essa conclusão ultilizando corelação). Ultilizei Tf-idf pois nesse caso palavrões não aparecem muitas vezes e por isso recebem peso maior.

**Inferencia** Os dados não estão presentes no dataset mais ainda sim o modelo acerta tudo. Podemos concluir que é um modelo praticamente perfeito

In [ ]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(regression, f)
with open('vectorinizer.pkl', 'wb') as f:
    vectorinizer.dump(regression, f)

In [ ]:
def is_valid_string(self,text):
        """
        Verifica se uma string é válida, ou seja, não é composta somente por espaços em branco,
        pontuações ou numerais.

        :param string: A string a ser verificada.
        :type string: str
        :return: Retorna True se a string é válida e False caso contrário.
        :rtype: bool
        """
        if text.strip() and not all(char.isdigit() or char.isspace() or char in string.punctuation for char in text):
            return True
        return False
class TokenCRIA():
    """
    Classe para padronizar o acesso a informações de tokens no formato CRIA

    Atributos:
    ----------
        texto : str
            Texto para compor Token
        span : tuple
            Tupla contendo posição inicial e final do texto na string original
    """
    def init(self,palavra,span):
        self.text = str(palavra)
        self.start = int(span[0])
        self.end = int(span[1])
        return

    def repr(self):
        return self.text

def divideSentencas(texto):
        """
        Divide as sentenças de um texto.
        ...

        Atributos:
        ---------
            texto : str
                String a ser dividida

        Retorna:
        ---------
                list[TokenCRIA] : list
                    Lista de TokenCRIA com sentenças do texto.
        """
        sentence_separator = '.?!'
        lista_expressoes_abreviadas = [
            r'\d',r'Art',r'Lei',r'N',r'Dr',r'Dra',
            r'Sr',r'Sra',r'Av',r'Sec',r'Apt']
        negative_lookbehind = '?<!'
        regular_exceptions = ''.join([f'({negative_lookbehind}{expressao})' for expressao in lista_expressoes_abreviadas])
        sentence_endings_pattern = f"{regular_exceptions}([{sentence_separator}])"
        splitStep = re.split(sentence_endings_pattern, texto)
        splitedList = [splitStep[index]+splitStep[index+1] for index in range(0,len(splitStep),2) if index!= len(splitStep)-1]
        sentences = [sentenca for sentenca in splitedList if is_valid_string(sentenca)]
        pos_finais = [texto.index(sentenca) + len(sentenca) for sentenca in sentences]
        pos_iniciais = [texto.index(sentenca) for sentenca in sentences]

divideSentencas('Querido filho,\n Fiquei sabendo que você terá seu primeiro debate fudido na próxima semana. Eu já conheço bem suas ideias antifeministas e sei que a feminista que debaterá com você é extremista. Sendo seu pai, sinto que é meu dever te lhe dar alguns conselhos.\n Quando for debater, mantenha a calma e não ataque a argumentadora oponente, mas sim seus argumentos. Esse é o primeiro passo para que o debate seja civilizado e produtivo. Use a razão e a lógica. Caso a feminista faça o oposto disso, e, por exemplo, descarte os seus argumentos meramente por você ser homem, branco, heterossexual e cristão, explique ao público que ela está usando a famosa falácia ad hominem.\n Evite e aponte, meu querido filho, todas as falácias lógicas. Falácias destroem o bom debate. Caso a feminista se esquive das críticas contra o feminismo, afirmando que aquilo que está sendo criticado por você chama-se "femismo" e não "feminismo", avise-a que ela está utilizando a falácia do escocês de verdade.\n  \n Mantenha-se firme também na defesa pela liberdade de expressão. Essa é outra regra de ouro para o bom debate. Não importa o quão verbalmente agressiva seja a feminista ou quão repulsivas pareçam ser suas ideias, lembre-se que ela possui o direito de falar tudo aquilo que realmente pensa. Se tudo o que ela fala for de fato tão horrível e irracional assim, então será fácil para você desconstruir publicamente os argumentos dela. Lembre-se daquilo que sempre te ensinei: tentar calar o debatedor adversário à força, como se estivesse cometendo um crime por expor o que pensa, é para aqueles que não conseguem convencer os demais através do uso da lógica e da razão, então precisam fazer isso através de meios coercitivos.\n Portanto, filho, se alguém nesse debate utilizar constantemente argumentos falaciosos ou tentar calar à força o adversário, alegando estar combatendo um "discurso de ódio", então que esse alguém não seja você. Não podemos controlar a feminista com quem você debaterá, mas você tem controle sobre si próprio. Faça a sua parte e tenha um bom debate !\n Com carinho,\n Seu amado Pai.')